In [1]:
import numpy as np
from sklearn import tree


#逻辑回归的学习率、权重的大小，影响整体收敛的快慢
#初始权重很重要


# H 测试样本分类结果
# TrainS 目标域样本
# TrainA 源域样本
# LabelS 目标域标签
# LabelA 源域标签
# Test  测试样本
# N 迭代次数


#计算weight
def calculate_P(weights, label):
    total = np.sum(weights)
    return np.asarray(weights / total, order='C')


#用逻辑回归作为基分类器，输出概率
def train_classify(trans_data, trans_label, test_data, P):
    clf = LogisticRegression(C=0.3,class_weight = 'balanced',solver='liblinear')
    clf.fit(trans_data, trans_label, sample_weight=P[:, 0])
    return clf.predict_proba(test_data)[:,1],clf


#计算在目标域上面的错误率
def calculate_error_rate(label_R, label_H, weight):
    total = np.sum(weight)
    return np.sum(weight[:, 0] / total * np.abs(label_R - label_H))


#根据逻辑回归输出的score的得到标签，注意这里不能用predict直接输出标签
def put_label(score_H,thred):
    new_label_H = []
    for i in score_H:
        if i <= thred:
            new_label_H.append(0)
        else:
            new_label_H.append(1)
    return new_label_H




NameError: name 'trans_A' is not defined

In [ ]:
#指定迭代次数，相当于集成模型中基模型的数量
N=500
trans_data = np.concatenate((trans_A, trans_S), axis=0)
trans_label = np.concatenate((label_A, label_S), axis=0)


row_A = trans_A.shape[0]
row_S = trans_S.shape[0]
row_T = test.shape[0]


test_data = np.concatenate((trans_data, test), axis=0)


# 初始化权重
weights_A = np.ones([row_A, 1])/row_A
weights_S = np.ones([row_S, 1])/row_S*2
weights = np.concatenate((weights_A, weights_S), axis=0)


bata = 1 / (1 + np.sqrt(2 * np.log(row_A / N)))


# 存储每次迭代的标签和bata值？
bata_T = np.zeros([1, N])  # 存每一次迭代的 error_rate / (1 - error_rate)
result_label = np.ones([row_A + row_S + row_T, N])


predict = np.zeros([row_T])


trans_data = np.asarray(trans_data, order='C')
trans_label = np.asarray(trans_label, order='C')
test_data = np.asarray(test_data, order='C')


best_ks = -1    #最优KS
best_round = -1 #最优基模型数量
best_model = -1 #最优模型


# 初始化结束


for i in range(N):
    P = calculate_P(weights, trans_label)


    result_label[:, i],model = train_classify(trans_data, trans_label,
                                        test_data, P)
    score_H = result_label[row_A:row_A + row_S, i]
    pctg = np.sum(data.bad_ind)/len(data.bad_ind)
    thred = pd.DataFrame(score_H).quantile(1-pctg)[0]
    label_H = put_label(score_H,thred)
    error_rate = calculate_error_rate(label_S, label_H,
                                      weights[row_A:row_A + row_S, :])


    if error_rate > 0.5:
        error_rate = 0.5
    if error_rate == 0:
        N = i
        break  # 防止过拟合
        # error_rate = 0.001


    bata_T[0, i] = error_rate / (1 - error_rate)


    # 调整目标域样本权重
    for j in range(row_S):
        weights[row_A + j] = weights[row_A + j] * np.power(bata_T[0, i],
                                                           (-np.abs(result_label[row_A + j, i] - label_S[j])))


    # 调整源域样本权重
    for j in range(row_A):
        weights[j] = weights[j] * np.power(bata, np.abs(result_label[j, i] - label_A[j]))


        
    y_pred = result_label[(row_A + row_S):,i]
    fpr_lr_train,tpr_lr_train,_ = roc_curve(val_y,y_pred)
    train_ks = abs(fpr_lr_train - tpr_lr_train).max()
    print('test_ks : ',train_ks,'当前第',i+1,'轮')
    
    
    if train_ks > best_ks :
        best_ks = train_ks
        best_round = i
        best_model = model